<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Creating-Non-Rectilinear-Grids-from-Scratch" data-toc-modified-id="Creating-Non-Rectilinear-Grids-from-Scratch-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Creating Non Rectilinear Grids from Scratch<a id="top"></a></a></span></li><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Table of Contents</a></span></li><li><span><a href="#Preparing-The-Notebook" data-toc-modified-id="Preparing-The-Notebook-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Preparing The Notebook<a id="prepare"></a></a></span></li><li><span><a href="#Curvilinear-Grids" data-toc-modified-id="Curvilinear-Grids-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Curvilinear Grids<a id="curv"></a></a></span><ul class="toc-item"><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Load the data<a id="load_curv"></a></a></span></li><li><span><a href="#Creating-latitude-and-longitude-two-dimensional-&quot;axes&quot;" data-toc-modified-id="Creating-latitude-and-longitude-two-dimensional-&quot;axes&quot;-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Creating latitude and longitude two dimensional "axes"<a id="axes_curv"></a></a></span></li><li><span><a href="#Creating-two-dimensional-curvilinear-grid" data-toc-modified-id="Creating-two-dimensional-curvilinear-grid-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Creating two dimensional curvilinear grid<a id="grid_curv"></a></a></span></li><li><span><a href="#Applying-grid-to-raw-data" data-toc-modified-id="Applying-grid-to-raw-data-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Applying grid to raw data<a id="data_curv"></a></a></span></li><li><span><a href="#Converting-to-another-grid" data-toc-modified-id="Converting-to-another-grid-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Converting to another grid<a id="regrid_curv"></a></a></span></li><li><span><a href="#Display-results" data-toc-modified-id="Display-results-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Display results<a id="plot_curv"></a></a></span></li></ul></li><li><span><a href="#Generic-Grids" data-toc-modified-id="Generic-Grids-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generic Grids<a id="gen"></a></a></span><ul class="toc-item"><li><span><a href="#Load-the-data" data-toc-modified-id="Load-the-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Load the data<a id="load_gen"></a></a></span></li><li><span><a href="#Creating-latitude-and-longitude--&quot;axes&quot;" data-toc-modified-id="Creating-latitude-and-longitude--&quot;axes&quot;-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Creating latitude and longitude  "axes"<a id="axes_gen"></a></a></span></li><li><span><a href="#Creating-Unidimensional-Generic-Grid" data-toc-modified-id="Creating-Unidimensional-Generic-Grid-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Creating Unidimensional Generic Grid<a id="grid_gen"></a></a></span></li><li><span><a href="#Applying-grid-to-raw-data" data-toc-modified-id="Applying-grid-to-raw-data-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Applying grid to raw data<a id="data_gen"></a></a></span></li><li><span><a href="#Display-results" data-toc-modified-id="Display-results-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Display results<a id="plot_gen"></a></a></span></li></ul></li></ul></div>

# Creating Non Rectilinear Grids from Scratch<a id="top"></a>

Download the Notebook [here](Creating_Non_Rectiilinear_Grids_From_Scratch.ipynb)

# Table of Contents

- [Preparing the notebook](#prepare)
- [Curvilinear Grid](#curv)
  - [Load in data](#load_curv)
  - [Prepare 2D "axes"](#axes_curv)
  - [Create Curvilinear Grid](#grid_curv)
  - [Applying Grid To Data](#data_curv)
  - [Regrid to T42 Gaussian Grid](#regrid_curv)
  - [Display Results](#plot_curv)
- [Generic Grid](#gen)
  - [Load in data](#load_gen)
  - [Prepare "axes"](#axes_gen)
  - [Create Generic Grid](#grid_gen)
  - [Applying Grid To Data](#data_gen)
  - [Display Results](#plot_gen)


[Back To Top](#top)

# Preparing The Notebook<a id="prepare"></a>

In [ ]:
# Prepare the notebook elements
import requests
def download(fnm):
    r = requests.get("https://uvcdat.llnl.gov/cdat/sample_data/%s" % fnm,stream=True)
    with open(fnm,"wb") as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter local_filename keep-alive new chunks
                f.write(chunk)
                
download("sampleCurveGrid4.nc")
download("sampleGenGrid3.nc")

# Curvilinear Grids<a id="curv"></a>

In this section we will create a curvilinear grid from raw data and regrid it to T42 Gaussian Grid

[Back To Top](#top)

## Load the data<a id="load_curv"></a>

[Back To Top](#top)

In [ ]:
import cdms2

fnm = "sampleCurveGrid4.nc"
f = cdms2.open(fnm)
data = f("sample").filled() # Get data and forget all about it (pure numpy)
lat = f("lat").filled() # latitude coordinates
lon = f("lon").filled() # longitude coords
blon = f("bounds_lon").filled() # longitude vertices
blat = f("bounds_lat").filled() #latitude vertices

## Creating latitude and longitude two dimensional "axes"<a id="axes_curv"></a>

Let's prepare the "axes" for our data (i.e 2D lat lon)

[Back To Top](#top)

In [ ]:
lat_axis = cdms2.coord.TransientAxis2D(lat)
lat_axis.setBounds(blat)

lon_axis = cdms2.coord.TransientAxis2D(lon)
lon_axis.setBounds(blon)

## Creating two dimensional curvilinear grid<a id="grid_curv"></a>

Let's prepare the "axes" for our data (i.e 2D lat lon)

[Back To Top](#top)

In [ ]:
# Now let's create the grid
grid = cdms2.hgrid.TransientCurveGrid(lat_axis,lon_axis)

## Applying grid to raw data<a id="data_curv"></a>

Let's apply the grid to our data

[Back To Top](#top)

In [ ]:
data = cdms2.MV2.array(data)
data.setAxisList(grid.getAxisList())
data.setGrid(grid)

## Converting to another grid<a id="regrid_curv"></a>

Let's regrid our data to a T42 grid

[Back To Top](#top)

In [ ]:
regrid_data = data.regrid(cdms2.createGaussianGrid(128),regridTool="esmf",regridMethod="conserve")

## Display results<a id="plot_curv"></a>

let's plot the raw data and the regrided data

[Back To Top](#top)

In [ ]:
# And llet's plot it to be sure it worked
import vcs
import EzTemplate

M = EzTemplate.Multi(rows=1,columns=2)

# Raw data
mesh = vcs.createmeshfill()
x=vcs.init(bg=True)
x.plot(data,mesh,M.get(row=0,column=0))

# Regridded data
x.plot(regrid_data,M.get(row=0,column=1))

# Generic Grids<a id="gen"></a>

In this section we will create a genric grid from raw data and regrid it to T42 Gaussian Grid

[Back To Top](#top)

## Load the data<a id="load_gen"></a>

[Back To Top](#top)

In [ ]:
import cdms2

fnm = "sampleGenGrid3.nc"
f = cdms2.open(fnm)
data = f("sample").filled() # Get data and forget all about it (pure numpy)
lat = f("lat").filled() # latitude coordinates
lon = f("lon").filled() # longitude coords
blon = f("bounds_lon").filled() # longitude vertices
blat = f("bounds_lat").filled() #latitude vertices

## Creating latitude and longitude  "axes"<a id="axes_gen"></a>

Let's prepare the "axes" for our data (i.e 1D lat lon)

[Back To Top](#top)

In [ ]:
#Ncells = len(lat)
#iaxis = cdms2.coord.TransientVirtualAxis("cell", Ncells)
lat_axis = cdms2.auxcoord.TransientAuxAxis1D(lat)
lat_axis.setBounds(blat)

lon_axis = cdms2.auxcoord.TransientAuxAxis1D(lon)
lon_axis.setBounds(blon)

## Creating Unidimensional Generic Grid<a id="grid_gen"></a>

Let's prepare the "axes" for our data (i.e 2D lat lon)

[Back To Top](#top)

In [ ]:
# Now let's create the grid
grid = cdms2.gengrid.TransientGenericGrid(lat_axis,lon_axis)

## Applying grid to raw data<a id="data_gen"></a>

Let's apply the grid to our data

[Back To Top](#top)

In [ ]:
data = cdms2.MV2.array(data)
data.setAxisList(grid.getAxisList())
data.setGrid(grid)

## Display results<a id="plot_gen"></a>

let's plot the raw data and the regrided data

[Back To Top](#top)

In [ ]:
# And llet's plot it to be sure it worked
import vcs

# Raw data
mesh = vcs.createmeshfill()
mesh.mesh = True
x=vcs.init(bg=True)
x.plot(data,mesh)

[Back To Top](#top)